**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(FD_RES, "source", "ENCODE_E2G", "annotation_e2g_benchmark_ENCSR998YDI")
dir(fdiry)

[1] "distal_regulation_combined_crispr_data_methods-1.pdf"
[2] "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.tsv"

## Import data

In [3]:
fdiry = file.path(FD_RES, "source", "ENCODE_E2G", "annotation_e2g_benchmark_ENCSR998YDI")
fname = "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

dat_peak_encode_e2g_import = dat
print(dim(dat))
head(dat)

[1] 10412    28


chrom,chromStart,chromEnd,name,EffectSize,strandPerturbationTarget,PerturbationTargetID,chrTSS,startTSS,endTSS,⋯,pValue,pValueAdjusted,PowerAtEffectSize25,PowerAtEffectSize10,PowerAtEffectSize15,PowerAtEffectSize20,PowerAtEffectSize50,ValidConnection,Notes,Reference
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,-0.293431866,.,chr1:3691278-3691778:.,chr1,3857213,3857214,⋯,NA,0.004023984,0.8250936,NA,NA,NA,NA,TRUE,Dataset: Fulco2019,"Ulirsch et al., 2016"
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,-0.331178093,.,chr1:3691278-3691778:.,chr1,3796503,3796504,⋯,NA,0.007771168,0.6089942,NA,NA,NA,NA,TRUE,Dataset: Fulco2019,"Ulirsch et al., 2016"
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,-0.472019217,.,chr1:3691278-3691778:.,chr1,3772761,3772762,⋯,NA,0.000641649,0.6393323,NA,NA,NA,NA,TRUE,Dataset: Fulco2019,"Ulirsch et al., 2016"
chr1,3803570,3805848,LRRC47|chr1:3720134-3722412:.,-0.001471265,.,chr1:3720134-3722412:.,chr1,3796503,3796504,⋯,0.6818652,0.999918431,1.0000000,0.40,0.80,0.85,1,TRUE,Dataset: Gasperini2019,"Gasperini et al., 2019"
chr1,3803570,3805848,SMIM1|chr1:3720134-3722412:.,0.025676924,.,chr1:3720134-3722412:.,chr1,3772788,3772789,⋯,0.3199596,0.994159681,1.0000000,0.80,0.85,1.00,1,TRUE,Dataset: Gasperini2019,"Gasperini et al., 2019"
chr1,4126791,4127291,SMIM1|chr1:4186851-4187351:.,0.023383787,.,chr1:4186851-4187351:.,chr1,3772788,3772789,⋯,0.3058098,0.992197132,0.9500000,0.55,0.80,0.90,1,TRUE,Dataset: Gasperini2019,"Gasperini et al., 2019"


In [4]:
for (txt in colnames(dat)){cat(txt, "\n")}

chrom 
chromStart 
chromEnd 
name 
EffectSize 
strandPerturbationTarget 
PerturbationTargetID 
chrTSS 
startTSS 
endTSS 
strandGene 
EffectSize95ConfidenceIntervalLow 
EffectSize95ConfidenceIntervalHigh 
measuredGeneSymbol 
measuredEnsemblID 
guideSpacerSeq 
guideSeq 
Significant 
pValue 
pValueAdjusted 
PowerAtEffectSize25 
PowerAtEffectSize10 
PowerAtEffectSize15 
PowerAtEffectSize20 
PowerAtEffectSize50 
ValidConnection 
Notes 
Reference 


## Arrange

In [5]:
dat = dat_peak_encode_e2g_import
dat = dat %>% 
    dplyr::rename(
        Chrom = chrom, 
        Start = chromStart, 
        End   = chromEnd, 
        Name  = name)

txt = "ENCODE-E2G_Benchmark"
dat = dat %>% 
    tidyr::separate(Name, c("Gene_Symbol", "Loc"), sep="\\|", remove = FALSE) %>% 
    dplyr::mutate(
        EffectSize   = round(EffectSize, 5),
        Region       = paste0(Chrom,  ":", Start,    "-", End),
        Gene_TSS     = paste0(chrTSS, ":", startTSS, "-", endTSS),
        Assay_Type   = txt,
        Assay_Label  = paste0(txt, ":", Gene_Symbol)
    )

vec = c('Chrom','Start','End','Name','EffectSize', 
        "Region", "Gene_Symbol", "Gene_TSS", 
        'Significant', 
        'Reference', 
        'Assay_Type', 'Assay_Label')
dat = dat %>% 
    dplyr::select({{vec}}) %>% 
    dplyr::distinct()
vec = dat$Gene_Symbol


dat_peak_encode_e2g_arrange = dat
cat("#Row: ", nrow(dat), "\n")
cat("#Gene:", length(unique(vec)), "\n")
head(dat)

#Row:  10412 
#Gene: 2146 


Chrom,Start,End,Name,EffectSize,Region,Gene_Symbol,Gene_TSS,Significant,Reference,Assay_Type,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,-0.29343,chr1:3774714-3775214,CEP104,chr1:3857213-3857214,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:CEP104
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,-0.33118,chr1:3774714-3775214,LRRC47,chr1:3796503-3796504,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,-0.47202,chr1:3774714-3775214,SMIM1,chr1:3772761-3772762,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1
chr1,3803570,3805848,LRRC47|chr1:3720134-3722412:.,-0.00147,chr1:3803570-3805848,LRRC47,chr1:3796503-3796504,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47
chr1,3803570,3805848,SMIM1|chr1:3720134-3722412:.,0.02568,chr1:3803570-3805848,SMIM1,chr1:3772788-3772789,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1
chr1,4126791,4127291,SMIM1|chr1:4186851-4187351:.,0.02338,chr1:4126791-4127291,SMIM1,chr1:3772788-3772789,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1


## Define active and inactive regions

In [6]:
dat = dat_peak_encode_e2g_arrange
dat = dat %>% 
    dplyr::filter(Significant) %>%
    dplyr::select(-Significant) %>%
    dplyr::arrange(Chrom, Start, End)

dat_peak_encode_e2g_active = dat
print(dim(dat))
head(dat)

[1] 569  11


Chrom,Start,End,Name,EffectSize,Region,Gene_Symbol,Gene_TSS,Reference,Assay_Type,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,-0.29343,chr1:3774714-3775214,CEP104,chr1:3857213-3857214,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:CEP104
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,-0.33118,chr1:3774714-3775214,LRRC47,chr1:3796503-3796504,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,-0.47202,chr1:3774714-3775214,SMIM1,chr1:3772761-3772762,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1
chr1,8858063,8858563,ENO1|chr1:8918122-8918622:.,-0.10741,chr1:8858063-8858563,ENO1,chr1:8879248-8879249,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:ENO1
chr1,17441310,17441810,SDHB|chr1:17767806-17768306:.,-0.08968,chr1:17441310-17441810,SDHB,chr1:17054169-17054170,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SDHB
chr1,17487740,17488240,RP4-798A10.7|chr1:17814236-17814736:.,0.03966,chr1:17487740-17488240,RP4-798A10.7,chr1:16514645-16514646,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:RP4-798A10.7


In [7]:
dat = dat_peak_encode_e2g_active
dat = dat %>%
    dplyr::select(Chrom, Start, End) %>%
    dplyr::distinct()

print(dim(dat))

[1] 496   3


In [8]:
vec_txt_region_total    = unique(dat_peak_encode_e2g_arrange$Region)
vec_txt_region_active   = unique(dat_peak_encode_e2g_active$Region)
vec_txt_region_inactive = setdiff(vec_txt_region_total, vec_txt_region_active)

dat = dat_peak_encode_e2g_arrange
dat = dat %>% 
    dplyr::filter(!Significant) %>%
    dplyr::filter(Region %in% vec_txt_region_inactive) %>%
    dplyr::select(Chrom, Start, End) %>%
    dplyr::distinct() %>%
    dplyr::arrange(Chrom, Start, End)

dat_peak_encode_e2g_inactive = dat
print(dim(dat))
head(dat)

[1] 3465    3


Chrom,Start,End
<chr>,<dbl>,<dbl>
chr1,3803570,3805848
chr1,4126791,4127291
chr1,5304578,5305078
chr1,8197448,8198244
chr1,8899850,8900350
chr1,9145155,9145655


In [9]:
dat = dat_peak_encode_e2g_arrange
dat = dat %>% 
    dplyr::select(Chrom, Start, End) %>%
    dplyr::distinct() %>%
    dplyr::arrange(Chrom, Start, End)

dat_peak_encode_e2g_total = dat
print(dim(dat))
head(dat)

[1] 3961    3


Chrom,Start,End
<chr>,<dbl>,<dbl>
chr1,3774714,3775214
chr1,3803570,3805848
chr1,4126791,4127291
chr1,5304578,5305078
chr1,8197448,8198244
chr1,8858063,8858563


## Data dictionary

```
dat_cnames = tribble(
    ~Name,         ~Description,
    "Chrom",       "Chromosome of the perturbed element",
    "Start",       "Start of the perturbed element (0-based)",
    "End",         "End of the perturbed element",
    "Name",        "Identifier for the given E-G pair in the form of 'gene|chr:start-stop:strand', e.g. 'BET1L|chr11:220536-220704:*'",
    "Score",       "Change in gene expression upon CRISPRi perturbation, e.g. precentage change in expression or log fold change",
    "Region",      "Location of the perturbed element",
    "Gene_Symbol", "target gene symbol",
    "Gene_TSS",    "target gene TSS",
    "Reference",   "Study from which this E-G pair was collected, e.g. Fulco2019",
    "Assay_Type",  "Assay type",
    "Assay_Label", "Assay label"
)
dat_cnames
```

## Interact for active regions

In [10]:
txt = c('chrom',  'chromStart', 'chromEnd',
        "chrTSS", "startTSS",   "endTSS",
        "name", 
        "EffectSize", 
        "Strand1", "Strand2",
        "Gene_Symbol", "Gene_TSS", "Reference")


dat = dat_peak_encode_e2g_import
dat = dat %>% 
    dplyr::filter(Significant) %>%
    tidyr::separate(name, c("Gene_Symbol", "Loc"), sep="\\|", remove = FALSE) %>% 
    dplyr::mutate(
        EffectSize   = round(EffectSize, 5),
        Gene_TSS     = paste0(chrTSS, ":", startTSS, "-", endTSS),
        Strand1      = ".",
        Strand2      = "."
    )

dat = dat %>% 
    dplyr::select({{txt}}) %>% 
    dplyr::arrange(chrom, chromStart, chromEnd)

dat_peak_encode_e2g_bedpe = dat
print(dim(dat))
head(dat)

[1] 569  13


chrom,chromStart,chromEnd,chrTSS,startTSS,endTSS,name,EffectSize,Strand1,Strand2,Gene_Symbol,Gene_TSS,Reference
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,3774714,3775214,chr1,3857213,3857214,CEP104|chr1:3691278-3691778:.,-0.29343,.,.,CEP104,chr1:3857213-3857214,"Ulirsch et al., 2016"
chr1,3774714,3775214,chr1,3796503,3796504,LRRC47|chr1:3691278-3691778:.,-0.33118,.,.,LRRC47,chr1:3796503-3796504,"Ulirsch et al., 2016"
chr1,3774714,3775214,chr1,3772761,3772762,SMIM1|chr1:3691278-3691778:.,-0.47202,.,.,SMIM1,chr1:3772761-3772762,"Ulirsch et al., 2016"
chr1,8858063,8858563,chr1,8879248,8879249,ENO1|chr1:8918122-8918622:.,-0.10741,.,.,ENO1,chr1:8879248-8879249,"Gasperini et al., 2019"
chr1,17441310,17441810,chr1,17054169,17054170,SDHB|chr1:17767806-17768306:.,-0.08968,.,.,SDHB,chr1:17054169-17054170,"Gasperini et al., 2019"
chr1,17487740,17488240,chr1,16514645,16514646,RP4-798A10.7|chr1:17814236-17814736:.,0.03966,.,.,RP4-798A10.7,chr1:16514645-16514646,"Gasperini et al., 2019"


In [11]:
dat = dat_peak_encode_e2g_bedpe

dat = dat %>%
    dplyr::mutate(score = 1000, value = 1, exp = ".", color = ".") %>%
    dplyr::mutate(
        chromMid = as.integer((chromStart + chromEnd) / 2),
        sourceName = paste0(chrom, ":", chromStart, "-", chromEnd),
        targetName = Gene_TSS,
    ) %>%
    dplyr::mutate(
        sourceChrom   = chrom,
        sourceStart   = chromStart,
        sourceEnd     = chromEnd,
        sourceStrand  = ".",
        targetChrom   = chrTSS,
        targetStart   = startTSS,
        targetEnd     = endTSS,
        targetStrand  = "."
    ) %>%
    dplyr::select(
        chrom, chromStart, chromEnd, name, 
        score, value, exp, color,
        sourceChrom,
        sourceStart,
        sourceEnd,
        sourceName,
        sourceStrand,
        targetChrom,
        targetStart,
        targetEnd,
        targetName,
        targetStrand
    )

dat_peak_encode_e2g_interact = dat
print(dim(dat))
head(dat)

[1] 569  18


chrom,chromStart,chromEnd,name,score,value,exp,color,sourceChrom,sourceStart,sourceEnd,sourceName,sourceStrand,targetChrom,targetStart,targetEnd,targetName,targetStrand
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3857213,3857214,chr1:3857213-3857214,.
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3796503,3796504,chr1:3796503-3796504,.
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3772761,3772762,chr1:3772761-3772762,.
chr1,8858063,8858563,ENO1|chr1:8918122-8918622:.,1000,1,.,.,chr1,8858063,8858563,chr1:8858063-8858563,.,chr1,8879248,8879249,chr1:8879248-8879249,.
chr1,17441310,17441810,SDHB|chr1:17767806-17768306:.,1000,1,.,.,chr1,17441310,17441810,chr1:17441310-17441810,.,chr1,17054169,17054170,chr1:17054169-17054170,.
chr1,17487740,17488240,RP4-798A10.7|chr1:17814236-17814736:.,1000,1,.,.,chr1,17487740,17488240,chr1:17487740-17488240,.,chr1,16514645,16514646,chr1:16514645-16514646,.


## Save results

```
K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.tsv

K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.total.bed.gz
K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.active.bed.gz
K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.inactive.bed.gz
```

In [12]:
FD_OUT = file.path(FD_RES, "results", "region", "annotation_encode_e2g_benchmark")

```
fdiry = FD_OUT
fname = "description.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_cnames
write_tsv(dat, fpath)
```

In [18]:
fdiry = FD_OUT
fname = "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.total.bed.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_total
#colnames(dat)[1] = "#Chrom"
write_tsv(dat, fpath, col_names = FALSE)
head(dat)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g_benchmark/K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.total.bed.gz"


Chrom,Start,End
<chr>,<dbl>,<dbl>
chr1,3774714,3775214
chr1,3803570,3805848
chr1,4126791,4127291
chr1,5304578,5305078
chr1,8197448,8198244
chr1,8858063,8858563


In [19]:
fdiry = FD_OUT
fname = "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.active.bed.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_active
#colnames(dat)[1] = "#Chrom"
write_tsv(dat, fpath, col_names = FALSE)
head(dat)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g_benchmark/K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.active.bed.gz"


Chrom,Start,End,Name,EffectSize,Region,Gene_Symbol,Gene_TSS,Reference,Assay_Type,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,-0.29343,chr1:3774714-3775214,CEP104,chr1:3857213-3857214,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:CEP104
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,-0.33118,chr1:3774714-3775214,LRRC47,chr1:3796503-3796504,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,-0.47202,chr1:3774714-3775214,SMIM1,chr1:3772761-3772762,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1
chr1,8858063,8858563,ENO1|chr1:8918122-8918622:.,-0.10741,chr1:8858063-8858563,ENO1,chr1:8879248-8879249,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:ENO1
chr1,17441310,17441810,SDHB|chr1:17767806-17768306:.,-0.08968,chr1:17441310-17441810,SDHB,chr1:17054169-17054170,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SDHB
chr1,17487740,17488240,RP4-798A10.7|chr1:17814236-17814736:.,0.03966,chr1:17487740-17488240,RP4-798A10.7,chr1:16514645-16514646,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:RP4-798A10.7


In [20]:
fdiry = FD_OUT
fname = "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.inactive.bed.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_inactive
#colnames(dat)[1] = "#Chrom"
write_tsv(dat, fpath, col_names = FALSE)
head(dat)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g_benchmark/K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.inactive.bed.gz"


Chrom,Start,End
<chr>,<dbl>,<dbl>
chr1,3803570,3805848
chr1,4126791,4127291
chr1,5304578,5305078
chr1,8197448,8198244
chr1,8899850,8900350
chr1,9145155,9145655


In [21]:
fdiry = FD_OUT
fname = "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.active.region2TSS.bedpe.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_bedpe
colnames(dat)[1] = "#chrom"
write_tsv(dat, fpath)
head(dat)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g_benchmark/K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.active.region2TSS.bedpe.gz"


#chrom,chromStart,chromEnd,chrTSS,startTSS,endTSS,name,EffectSize,Strand1,Strand2,Gene_Symbol,Gene_TSS,Reference
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,3774714,3775214,chr1,3857213,3857214,CEP104|chr1:3691278-3691778:.,-0.29343,.,.,CEP104,chr1:3857213-3857214,"Ulirsch et al., 2016"
chr1,3774714,3775214,chr1,3796503,3796504,LRRC47|chr1:3691278-3691778:.,-0.33118,.,.,LRRC47,chr1:3796503-3796504,"Ulirsch et al., 2016"
chr1,3774714,3775214,chr1,3772761,3772762,SMIM1|chr1:3691278-3691778:.,-0.47202,.,.,SMIM1,chr1:3772761-3772762,"Ulirsch et al., 2016"
chr1,8858063,8858563,chr1,8879248,8879249,ENO1|chr1:8918122-8918622:.,-0.10741,.,.,ENO1,chr1:8879248-8879249,"Gasperini et al., 2019"
chr1,17441310,17441810,chr1,17054169,17054170,SDHB|chr1:17767806-17768306:.,-0.08968,.,.,SDHB,chr1:17054169-17054170,"Gasperini et al., 2019"
chr1,17487740,17488240,chr1,16514645,16514646,RP4-798A10.7|chr1:17814236-17814736:.,0.03966,.,.,RP4-798A10.7,chr1:16514645-16514646,"Gasperini et al., 2019"


In [22]:
fdiry = FD_OUT
fname = "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.active.region2TSS.bed"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_interact
colnames(dat)[1] = "#chrom"
write_tsv(dat, fpath)
head(dat)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g_benchmark/K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.active.region2TSS.bed"


#chrom,chromStart,chromEnd,name,score,value,exp,color,sourceChrom,sourceStart,sourceEnd,sourceName,sourceStrand,targetChrom,targetStart,targetEnd,targetName,targetStrand
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3857213,3857214,chr1:3857213-3857214,.
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3796503,3796504,chr1:3796503-3796504,.
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3772761,3772762,chr1:3772761-3772762,.
chr1,8858063,8858563,ENO1|chr1:8918122-8918622:.,1000,1,.,.,chr1,8858063,8858563,chr1:8858063-8858563,.,chr1,8879248,8879249,chr1:8879248-8879249,.
chr1,17441310,17441810,SDHB|chr1:17767806-17768306:.,1000,1,.,.,chr1,17441310,17441810,chr1:17441310-17441810,.,chr1,17054169,17054170,chr1:17054169-17054170,.
chr1,17487740,17488240,RP4-798A10.7|chr1:17814236-17814736:.,1000,1,.,.,chr1,17487740,17488240,chr1:17487740-17488240,.,chr1,16514645,16514646,chr1:16514645-16514646,.
